In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
from linecache import getline
from matplotlib.patches import Ellipse
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import shapefile
import time
import os
import numpy.ma as ma
import sys
import shutil
import datetime
import rtnorm
import numpy as np
import numba
from numba import jit
import math 
from numba import vectorize, int64, float32, float64
from numba import cuda

In [ ]:
@cuda.jit
def ellipse_cuda( xc , yc , ax1 , ax2 , angle , X_circle , Y_circle, xe, ye ):
    i = cuda.grid(1)
    if i < X_circle.size:
        cos_angle = math.cos(angle*np.pi/180)
        sin_angle = math.sin(angle*np.pi/180)
        
        # X = ax1 * X_circle
        # Y = ax2 * Y_circle
        
        # xe = xc + X*cos_angle - Y*sin_angle
        # ye = yc + X*sin_angle + Y*cos_angle
        xe[i] = xc + ax1*X_circle[i]*cos_angle - ax2*Y_circle[i]*sin_angle
        ye[i] = yc + ax1*X_circle[i]*sin_angle + ax2*Y_circle[i]*cos_angle


xc = 288830.0
yc = 2150362.0
ax1 = 26.081118798477803
ax2 = 24.409220221209875
angle = 39.8274074028988
x_circle = np.full((30), 4.0)
y_circle = np.full((30), 3.0)

threads = x_circle.size
blocks = 1 

xe = np.zeros(x_circle.size)
ye = np.zeros(y_circle.size)


ellipse_cuda[threads,blocks]( xc , yc , ax1 , ax2 , angle , x_circle , y_circle, xe, ye )


In [ ]:
@jit(nopython=True)
def ellipse( xc , yc , ax1 , ax2 , angle , X_circle , Y_circle ):

    cos_angle = np.cos(angle*np.pi/180)
    sin_angle = np.sin(angle*np.pi/180)


    
    X = ax1 * X_circle
    Y = ax2 * Y_circle
    
    
    xe = xc + X*cos_angle - Y*sin_angle


    ye = yc + X*sin_angle + Y*cos_angle
    return (xe,ye)



xc = 288830.0
yc = 2150362.0
ax1 = 26.081118798477803
ax2 = 24.409220221209875
angle = 39.8274074028988
x_circle = np.full((30), 4.0)
y_circle = np.full((30), 3.0)


print(ellipse( xc , yc , ax1 , ax2 , angle , x_circle , y_circle ))

In [ ]:
# Times both the jit ellipse function and the cuda function
# timeit is however not really optimal to test between CPU and GPU functions
%timeit ellipse(xc, yc, ax1, ax2, angle, x_circle, y_circle)
%timeit ellipse_cuda(xc, yc, ax1, ax2, angle, x_circle, y_circle)
